In [1]:
pip install gurobipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Loading Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

## 1. Loading the data

In [3]:
#read csv into dataframe
carbon_emm = pd.read_csv('carbon_emissions.csv',index_col=0)
cost_years = pd.read_csv('cost_profiles.csv', index_col=0)
demand = pd.read_csv('demand.csv')
fuels = pd.read_csv('fuels.csv')
fuels['Comb'] = fuels['Fuel'] + fuels['Year'].astype(str)
fuel_sub = fuels[['Comb','Emissions (CO2/unit_fuel)']]
fuel_sub
vehicles = pd.read_csv('vehicles.csv') #192
vehicle_fuel = pd.read_csv('vehicles_fuels.csv') #320
vehicle_master = vehicles.merge(vehicle_fuel, left_on='ID', right_on='ID')
vehicle_master['Comb'] = vehicle_master['Fuel'] + vehicle_master['Year'].astype(str)
vehicle_master = vehicle_master.merge(fuel_sub, left_on='Comb', right_on='Comb')
vehicle_master

,ID,Vehicle,Size,Year,Cost ($),Yearly range (km),Distance,Fuel,Consumption (unit_fuel/km),Comb,Emissions (CO2/unit_fuel)
0,BEV_S1_2023,BEV,S1,2023,187000,102000,D1,Electricity,0.893043,Electricity2023,0.000000
1,BEV_S1_2024,BEV,S1,2024,177650,102000,D1,Electricity,0.893043,Electricity2024,0.000000
2,BEV_S1_2025,BEV,S1,2025,168767,102000,D1,Electricity,0.893043,Electricity2025,0.000000
3,BEV_S1_2026,BEV,S1,2026,160329,102000,D2,Electricity,0.893043,Electricity2026,0.000000
4,BEV_S1_2027,BEV,S1,2027,152312,102000,D2,Electricity,0.868161,Electricity2027,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
315,LNG_S3_2036,LNG,S3,2036,221948,73000,D4,BioLNG,0.155192,BioLNG2036,0.378439
316,LNG_S3_2037,LNG,S3,2037,228607,73000,D4,LNG,0.153945,LNG2037,2.486188
317,LNG_S3_2037,LNG,S3,2037,228607,73000,D4,BioLNG,0.155192,BioLNG2037,0.378439
318,LNG_S3_2038,LNG,S3,2038,235465,73000,D4,LNG,0.153945,LNG2038,2.486188


In [4]:
# Indexes
# t is current year
# p is purchae year
# s is size
# k is vehicle type is 8 (4 bev d1,d2,d3,d4 and 4 disel , lng, bio lng, bio diesel)

### Functions to call

In [5]:
def get_size_name(s):
    return 'S'+str(s+1)

In [6]:
def get_dist_name(d):
    return 'D'+str(d+1)

In [7]:
def get_year(t):
    return 2023+t

In [8]:
dict_of_k = {'LNG':1,'BioLNG':2,'HVO':3,'B20':4,'ElectricityD1':5,'ElectricityD2':6,'ElectricityD3':7,'ElectricityD4':8}

In [9]:
vehicle_master['k'] = vehicle_master['Consumption (unit_fuel/km)']
for i in range(320):
  if vehicle_master.iloc[i][7] == 'Electricity':
    vehicle_master.at[i,'k'] = dict_of_k[vehicle_master.iloc[i][7]+vehicle_master.iloc[i][6]]
  else:
    vehicle_master.at[i,'k'] = dict_of_k[vehicle_master.iloc[i][7]]
vehicle_master['k'] = vehicle_master['k'].astype(int)

C:\Users\ChemeGrad2021\AppData\Local\Temp\ipykernel_1544\157013792.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if vehicle_master.iloc[i][7] == 'Electricity':
C:\Users\ChemeGrad2021\AppData\Local\Temp\ipykernel_1544\157013792.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  vehicle_master.at[i,'k'] = dict_of_k[vehicle_master.iloc[i][7]+vehicle_master.iloc[i][6]]
C:\Users\ChemeGrad2021\AppData\Local\Temp\ipykernel_1544\157013792.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavi

In [10]:
def truck_cost(p,s,k):
    #age = t-p+1
    size = get_size_name(s)
    year = get_year(p)
    if vehicle_master[ (vehicle_master['Size'] == size) & (vehicle_master['k'] == k) & (vehicle_master['Year'] == year) ]['Cost ($)'].values.size == 0 :
        return 0
    else :
        cost = vehicle_master[ (vehicle_master['Size'] == size) & (vehicle_master['k'] == k) & (vehicle_master['Year'] == year) ]['Cost ($)'].values[0]
    return cost

In [11]:
def truck_insurance(t,p,s,k):
    age = t-p+1
    size = get_size_name(s)
    year = get_year(p)
    #print(t,p,s,k)
    if  vehicle_master[ (vehicle_master['Size'] == size) & (vehicle_master['k'] == k) & (vehicle_master['Year'] == year) ]['Cost ($)'].values.size == 0 or age > 10 or age <=0 :
        return 0
    else:
        insurance_cost = cost_years.loc[age][1] * 0.01 * vehicle_master[ (vehicle_master['Size'] == size) & (vehicle_master['k'] == k) & (vehicle_master['Year'] == year) ]['Cost ($)'].values[0]
    return insurance_cost

In [12]:
def truck_maintenance(t,p,s,k):
    age = t-p+1
    size = get_size_name(s)
    year = get_year(p)
    if  vehicle_master[ (vehicle_master['Size'] == size) & (vehicle_master['k'] == k) & (vehicle_master['Year'] == year) ]['Cost ($)'].values.size == 0 or age > 10 or age <=0:
        return 0
    else :
        mnt_cost = cost_years.loc[age][2] * 0.01 * vehicle_master[ (vehicle_master['Size'] == size) & (vehicle_master['k'] == k) & (vehicle_master['Year'] == year) ]['Cost ($)'].values[0]
    return mnt_cost

In [13]:
def truck_selling_price(t,p,s,k):
    age = t-p+1
    size = get_size_name(s)
    year = get_year(p)
    if vehicle_master[ (vehicle_master['Size'] == size) & (vehicle_master['k'] == k) & (vehicle_master['Year'] == year) ]['Cost ($)'].values.size == 0 or age > 10 or age <=0:
        return 0
    else:
        sp_price = cost_years.loc[age][0] * 0.01 * vehicle_master[ (vehicle_master['Size'] == size) & (vehicle_master['k'] == k) & (vehicle_master['Year'] == year) ]['Cost ($)'].values[0]
    return sp_price

In [14]:
def get_fuel_cons(s,k,p):
    size = get_size_name(s)
    year = get_year(p)
    if vehicle_master[ (vehicle_master['Size'] == size) & (vehicle_master['k'] == k) & (vehicle_master['Year'] == year) ]['Consumption (unit_fuel/km)'].values.size == 0 :
        return 0
    else:
       value = vehicle_master[ (vehicle_master['Size'] == size) & (vehicle_master['k'] == k) & (vehicle_master['Year'] == year) ]['Consumption (unit_fuel/km)'].values[0]
    return value

In [15]:
def get_range(s,k,p):
    size = get_size_name(s)
    year = get_year(p)
    if vehicle_master[ (vehicle_master['Size'] == size) & (vehicle_master['k'] == k) & (vehicle_master['Year'] == year) ]['Yearly range (km)'].values.size == 0 :
        return 0
    else :
        value = vehicle_master[ (vehicle_master['Size'] == size) & (vehicle_master['k'] == k) & (vehicle_master['Year'] == year) ]['Yearly range (km)'].values[0]
    return value

In [16]:
def get_emission_kg(k,t):
    year = get_year(t)
    if vehicle_master[ (vehicle_master['k'] == k) & (vehicle_master['Year'] == year) ]['Emissions (CO2/unit_fuel)'].values.size == 0:
        return 0
    else:
        value = vehicle_master[ (vehicle_master['k'] == k) & (vehicle_master['Year'] == year) ]['Emissions (CO2/unit_fuel)'].values[0]
    return value

In [17]:
def ret_fs(t,p,s,k):
    age = t-p+1
    size = get_size_name(s)
    year = get_year(p)
    if vehicle_master[ (vehicle_master['Size'] == size) & (vehicle_master['k'] == k) & (vehicle_master['Year'] == year) ]['Cost ($)'].values.size == 0 :
        return 1
    else :
        return 0

In [18]:
def truck_fuel_cost(t,p,s,k):
    age = t-p+1
    fuel_list = ['LNG','BioLNG','HVO','B20','Electricity']
    year = get_year(t)
    if k<=4:
        fuel_name = fuel_list[k-1]
    else:
        fuel_name = 'Electricity'
    if age > 10 or age <=0:
        return 0
    else:
        fuel_cost = get_fuel_cons(s,k,p) * get_range(s,k,p) * fuels[ (fuels['Fuel'] == fuel_name) & (fuels['Year'] == year) ]['Cost ($/unit_fuel)'].values[0] #
    return fuel_cost

In [19]:
vehicle_master.to_csv('vehicle_master.csv', index=False)

## 2. Optimization Section

### 2.1 Defining Indexes 

In [20]:
s = [0,1,2,3]
t = [i for i in range(0,16)]
t_h = t[:-1]
p = [i for i in range(0,16)]
k = [i for i in range(1,5)]
d = [i for i in range(0,4)]

In [21]:
start_values = pd.read_csv('warm_start_new.csv')
start_values

,Solver_Var,Solver_Value
0,"P[0,0,1]",72.0
1,"P[0,0,2]",0.0
2,"P[0,0,3]",0.0
3,"P[0,0,4]",1.0
4,"P[0,1,1]",30.0
...,...,...
41211,"F[15,15,2,4]",0.0
41212,"F[15,15,3,1]",0.0
41213,"F[15,15,3,2]",8.0
41214,"F[15,15,3,3]",2.0


In [22]:
with gp.Env(empty=True) as env:
    env.setParam("WLSACCESSID","cb05e01d-e51a-4aa7-ae0b-036f1fe35ef0")
    env.setParam("WLSSECRET","c6e9ce13-75d8-4aa4-a6a1-88b7a2b4b29d")
    env.setParam("LICENSEID",2525771)
    env.start()

    with gp.Model("Shell Hack", env=env) as m:

        ### VARIABLES DEFINED
        
        # Create integer variable to represent how many trucks are being bought in year t, purchase year p, size s, and vehicle type k
        P = m.addVars(t,s,k, lb=0, vtype=GRB.INTEGER, name="P") 
        #set variables in certain range of indexes ranges
        
        # Create an integer variable to represent how many trucks are being sold in year t, purchased in year p, size s, and vehicle type k
        S = m.addVars(t,p,s,k, lb = 0, vtype=GRB.INTEGER, name="S")

        # Create an integer variable to represent how many trucks are being used in year t, purchased in year p, of size s and vehicle type k 
        U = m.addVars(t,p,s,k,d, lb = 0, vtype=GRB.CONTINUOUS, name = "U")
        T = m.addVars(t,p,s,k,d, lb = 0, vtype=GRB.INTEGER, name = "T")

        # Create an continuous variable to check what percentage of trucks used are being utilized
        #D = m.addVars(t,p,s,k, vtype=GRB.CONTINUOUS, ub = 1, lb = 0, name = "D")

        ### Dummy variables

        # Fleet size variable and it is measured at the end of year tt
        F = m.addVars(t,p,s,k, vtype=GRB.INTEGER, lb = 0, name = "F" )

        # Parked fleet variable 
        #PK = m.addVars(t,p,s,k, vtype = GRB.INTEGER, lb=0, name = "PK")

        #############
        ### CONSTRAINTS DEFINED

        #1 Fleet size at the end of year 1 Purchase just before the sale of trucks
        m.addConstrs((F[0,0,ss,kk] == P[0,ss,kk] for ss in s for kk in k), name="c1")

        #2 Second year onwards fleet is last year fleet + new purchases - sold that year. 
        m.addConstrs((F[tt+1,pp,ss,kk] == F[tt,pp,ss,kk] - S[tt,pp,ss,kk] for ss in s for kk in k for tt in t_h for pp in range(tt+1)  ) , name = "c2")
        m.addConstrs((F[tt+1,tt+1,ss,kk] ==  P[tt+1,ss,kk]  for ss in s for kk in k for tt in t_h  ) , name = "c2.2")
        m.addConstrs((0 <= F[15,pp,ss,kk] - S[15,pp,ss,kk] for ss in s for kk in k for pp in range(16)) , name = "c2.3")
        
        #3 Every year, need to ensure parked = fleet - used

        m.addConstrs((F[tt,pp,ss,kk] >= T.sum(tt,pp,ss,kk,'*') for ss in s for kk in k for tt in t for pp in range(tt+1)  ) , name = "c3.2")
        m.addConstrs((U[tt,pp,ss,kk,dd] <= T[tt,pp,ss,kk,dd] for ss in s for kk in k for tt in t for pp in range(tt+1) for dd in range(4) ) , name = "c3.3" )

        #4 Age of vehicle is less than 10 years
        m.addConstrs((F[tt,pp,ss,kk] == 0 for pp in range(0,6) for ss in s for kk in k for tt in range(pp+10,16)) , name = "c4")  

        #5 Sales every year is less then 20% of fleet size
        m.addConstrs(( S.sum(tt,'*','*','*') <= 0.20 * F.sum(tt,'*','*','*') for tt in t ) , name = 'c5.1')

        #6 Emissions constraints : for every year t find sum of U and D and multiply by emissions of that year
        m.addConstrs(( gp.quicksum((U.sum(tt,pp,ss,kk,'*')) *  get_fuel_cons(ss,kk,pp) * get_range(ss,kk,pp) * get_emission_kg(kk,tt) for pp in range(tt+1) for ss in s for kk in k) <= (carbon_emm.iloc[tt].values[0]-1-1e-7)  for tt in t), name = 'c6')   #best came here 0.000001 and 1.5% Gap 
        
        #7 demand constraints
        #7.1 D4
        for tt in t:
            for ss in s:
                m.addConstr( (gp.quicksum(U[tt,pp,ss,kk,3] * get_range(ss,kk,pp)  for pp in range(max(tt-9,0),tt+1) for kk in k ) >= 1+1e-7 + demand[ (demand['Year']==2023+tt) & (demand['Size']==get_size_name(ss)) & (demand['Distance']=='D4') ]['Demand (km)'].values[0]), name = 'c7')
             
        #7.2 D3
        m.addConstrs ( (gp.quicksum( (U[tt,pp,ss,kk,2])  * get_range(ss,kk,pp) for pp in range(max(tt-9,0),tt+1) for kk in k) >= 1+1e-7 + demand[ (demand['Year']==2023+tt) & (demand['Size']==get_size_name(ss)) & (demand['Distance']=='D3') ]['Demand (km)'].values[0]  for tt in t for ss in s), name = 'c8')

        #7.3 D2
        m.addConstrs( (gp.quicksum( (U[tt,pp,ss,kk,1]) * get_range(ss,kk,pp) for pp in range(max(tt-9,0),tt+1) for kk in k) >= 1+1e-7 + (demand[ (demand['Year']==2023+tt) & (demand['Size']==get_size_name(ss)) & (demand['Distance']=='D2') ]['Demand (km)'].values[0]) for tt in t for ss in s), name = 'c9')

        #7.3 D1
        m.addConstrs( (gp.quicksum( (U[tt,pp,ss,kk,0]) * get_range(ss,kk,pp) for pp in range(max(tt-9,0),tt+1) for kk in k ) >= 1+1e-7 + (demand[ (demand['Year']==2023+tt) & (demand['Size']==get_size_name(ss)) & (demand['Distance']=='D1') ]['Demand (km)'].values[0]) for tt in t for ss in s), name = 'c10')       

        #8 when age is greater than = 10 force to be 0
        m.addConstrs((U[tt,pp,ss,kk,dd] == 0 for pp in t for tt in range(min(pp+9,16),16) for ss in s for kk in k for dd in range(4)), name = "c11.1")
        m.addConstrs((T[tt,pp,ss,kk,dd] == 0 for pp in t for tt in range(min(pp+9,16),16) for ss in s for kk in k for dd in range(4)), name = "c11.2")
  
#8 t > p make it 0 
        m.addConstrs((U[tt,pp,ss,kk,dd] == 0 for pp in t for tt in range(pp) for ss in s for kk in k for dd in range(4)), name = "c11.2")
        m.addConstrs((T[tt,pp,ss,kk,dd] == 0 for pp in t for tt in range(pp) for ss in s for kk in k for dd in range(4)), name = "c11.3")
        m.addConstrs((F[tt,pp,ss,kk] == 0 for pp in t for tt in range(pp) for ss in s for kk in k), name = "c13.2")
        m.addConstrs((S[tt,pp,ss,kk] == 0 for pp in t for tt in range(pp) for ss in s for kk in k), name = "c15.2")


        #9 checkiug if the vehicle exists in the first place
        m.addConstrs((P[tt,ss,kk] * ret_fs(tt,pp,ss,kk) == 0 for tt in t for pp in range(max(tt-9,0),tt+1) for ss in s for kk in k), name = "c16")
        
        #m.addConstrs( ((P[pp,ss,kk] == 0) >> (U[tt,pp,ss,kk,dd] == 0) for tt in t for pp in range(tt+1) for ss in s for kk in k for dd in range(4) ) , name = "c17")
        #m.addConstrs( ((P[pp,ss,kk] == 0) >> (S[tt,pp,ss,kk] == 0) for tt in t for pp in range(tt+1) for ss in s for kk in k ) , name = "c17.1")
        m.addConstrs( (F[tt,pp,ss,kk] <= P[pp,ss,kk] * 1e+6 for tt in t for pp in range(tt+1) for ss in s for kk in k ) , name = "c17.1")
        m.addConstrs( (S[tt,pp,ss,kk] <= P[pp,ss,kk] * 1e+6 for tt in t for pp in range(tt+1) for ss in s for kk in k ) , name = "c17.2")
        m.addConstrs( (U[tt,pp,ss,kk,dd] <= P[pp,ss,kk] * 1e+6 for tt in t for pp in range(tt+1) for ss in s for kk in k for dd in range(4) ) , name = "c17.3")
        m.addConstrs( (T[tt,pp,ss,kk,dd] <= P[pp,ss,kk] * 1e+6 for tt in t for pp in range(tt+1) for ss in s for kk in k for dd in range(4) ) , name = "c17.4")

        #10 constraint is 
        # m.addConstrs( (U[tt,pp,ss,kk,dd] == 0 for tt in t for pp in range(tt+1) for ss in s for kk in [5,6,7] for dd in range(kk-4,4) ), name = "c18")
        # m.addConstrs( (T[tt,pp,ss,kk,dd] == 0 for tt in t for pp in range(tt+1) for ss in s for kk in [5,6,7] for dd in range(kk-4,4) ), name = "c19")

        ##############
        # OBJECTIVE FUNCTION - - -add fuel to this. 
        m.setObjective(
            #gp.quicksum( P[tt,ss,kk] * truck_cost(tt,pp,ss,kk)   for tt in t for pp in range(max(tt-10,0),tt+1) for ss in s for kk in k) , GRB.MINIMIZE)
            gp.quicksum( P[tt,ss,kk] * truck_cost(tt,ss,kk) for tt in t for ss in s for kk in k) - gp.quicksum( S[tt,pp,ss,kk] * truck_selling_price(tt,pp,ss,kk) for tt in t for pp in range(tt+1) for ss in s for kk in k) \
            + gp.quicksum( F[tt,pp,ss,kk] * truck_insurance(tt,pp,ss,kk) for tt in t for pp in range(max(tt-10,0),tt+1) for ss in s for kk in k) \
            + gp.quicksum( F[tt,pp,ss,kk] * truck_maintenance(tt,pp,ss,kk) for tt in t for pp in range(max(tt-10,0),tt+1) for ss in s for kk in k)\
            + gp.quicksum( U.sum(tt,pp,ss,kk,'*') * truck_fuel_cost(tt,pp,ss,kk) for tt in t for pp in range(max(tt-9,0),tt+1) for ss in s for kk in k) \
            - gp.quicksum((F[15,pp,ss,kk]-S[15,pp,ss,kk]) * truck_selling_price(tt,pp,ss,kk) for pp in range(15+1) for ss in s for kk in k), GRB.MINIMIZE)

        # Update and write the model
        m.update() # Update model parameters
        m.write("Shell.lp") # Write model to file
        m.setParam('MIPGap', 0.001)
        m.setParam('Method', 4)
        m.setParam('Cuts', 2)
        m.setParam('Heuristics', 0.4)
        m.setParam('TimeLimit', 3600)
        m.setParam('MIPFocus', 2)
        m.Params.IntFeasTol = 1e-9

        #m.setParam('NonConvex', 2)
        # m.Params.ConcurrentMIP = 4
        # m.Params.MIPFocus = 2

        for v in m.getVars():
            if v.VarName in start_values['Solver_Var'].values:
                v.start = start_values[start_values['Solver_Var']==v.VarName]['Solver_Value'].values[0]

        m.optimize()
        print("\nObjective value: ", m.getAttr("ObjVal"))

        #write into csv with specific column names
        output = pd.DataFrame(columns=['Solver_Var','Solver_Value','Year','ID','Num_Vehicles','Type','Fuel','Distance_bucket','Distance_per_vehicle(km)'])
        warm_start = pd.DataFrame(columns=['Solver_Var','Solver_Value'])
        #make csv from output
        output.to_csv('output_new.csv', index=False)
        warm_start.to_csv('warm_start_new.csv', index=False)
        # append rows into csv

        # Print the optimal values of the decision variables and store in csv
        for v in m.getVars():
            warm_start.loc[len(warm_start)] = [v.VarName, v.X]
        warm_start.to_csv('warm_start_new.csv', mode='a', header=False, index=False)
        
        for v in m.getVars():
            if v.X > 0.00001:
                #print(v.VarName, v.X)
                #add to csv
                output.loc[len(output)] = [v.VarName, v.X,0,0,0,0,0,0,0]
        output.to_csv('output_new.csv', mode='a', header=False, index=False)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2525771
Academic license 2525771 - for non-commercial use only - registered to sh___@mit.edu


C:\Users\ChemeGrad2021\AppData\Local\Temp\ipykernel_1544\2723104624.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sp_price = cost_years.loc[age][0] * 0.01 * vehicle_master[ (vehicle_master['Size'] == size) & (vehicle_master['k'] == k) & (vehicle_master['Year'] == year) ]['Cost ($)'].values[0]
C:\Users\ChemeGrad2021\AppData\Local\Temp\ipykernel_1544\1055066232.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  insurance_cost = cost_years.loc[age][1] * 0.01 * vehicle_master[ (vehicle_master['Size'] == size) & (vehicle_master['k'] == k) & (vehicle_master['Year'] == year) ]['Cost ($)'].values[0]


Set parameter MIPGap to value 0.001
Set parameter Method to value 4
Set parameter Cuts to value 2
Set parameter Heuristics to value 0.4
Set parameter TimeLimit to value 3600
Set parameter MIPFocus to value 2
Set parameter IntFeasTol to value 1e-09
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2525771 - for non-commercial use only - registered to sh___@mit.edu
Optimize a model with 60320 rows, 41216 columns and 125968 nonzeros
Model fingerprint: 0x6c668db4
Variable types: 16384 continuous, 24832 integer (0 binary)
Coefficient statistics:
  Matrix range     [2e-01, 1e+06]
  Objective range  [4e-12, 3e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+03, 1e+07]

Loaded user MIP start with objective 1.79782e+08

Presolve removed 36817 rows and 24512 col